# Explore here

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from joblib import dump
import numpy as np
import gzip
import shutil

# Cargar los datos
data = pd.read_csv('anime.csv')

# Manejo de valores faltantes y no numéricos
data = data.dropna(subset=['rating'])  # Eliminar filas con valores nulos en 'rating'
data = data.reset_index(drop=True)

# Reemplazar 'Unknown' con NaN y eliminar esas filas
data.replace('Unknown', np.nan, inplace=True)
data.dropna(inplace=True)

# Codificación de las características categóricas
label_encoder = LabelEncoder()
data['type'] = label_encoder.fit_transform(data['type'])

# One-hot encoding para la columna 'genre'
genre_dummies = data['genre'].str.get_dummies(sep=', ')
data = pd.concat([data, genre_dummies], axis=1)

# Selección de las características y la variable objetivo
X = data.drop(columns=['name', 'genre'])
y = data['name']

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de clasificación
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Guardar el modelo y las características de entrenamiento
model_filename = "decision_tree_classifier_anime.joblib"
dump(model, model_filename)

# Comprimir el archivo guardado
with open(model_filename, 'rb') as f_in:
    with gzip.open(model_filename + '.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Opcional: guardar las características de entrenamiento en un CSV
X_train.columns.to_series().to_csv('train_features.csv', index=False)

# Eliminar el archivo original para ahorrar espacio (opcional)
import os
os.remove(model_filename)
